In [8]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_rows", 15, "display.max_columns", None)
os.getcwd()
df = pd.read_csv('./dws_dsc_customer_opportunity_monthly_kpi_1.csv')
op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
op.columns = [re.sub('\w+\.', '', i) for i in list(op.columns)]
df = df.fillna(0)
# df.columns
df['if_cooped'] = 0
df['if_cooped'] = df['if_cooped'].where(df['contract_signed_opportunity_num'] < 1, 1) # 今年有没合作。
cus_bf_21 = op[(op['createdon'] < '2021-11-01') & (op['milestone'] == 'Contract Signed')]['account_name_en'].unique()
cus_bf_21 = list(cus_bf_21)
cus_af_21 = list(df['customer_name'].unique())
df['old_cust'] = df['customer_name'].isin(cus_bf_21)
# df.to_csv('./data_down/dws_dsc_customer_opportunity_monthly_kpi_1.csv', index = None, encoding='utf_8_sig')

df['annual_average_revenue'] = df['annual_average_revenue'] * 1000
df['annual_average_gross_profit'] = df['annual_average_gross_profit'] * 1000

op = op[op['annual_average_gross_profit'].str.match('(\d+\.)').fillna(False)]
op = op[op['annual_average_revenue'].str.match('(\d+\.)').fillna(False)]

op['annual_average_revenue'] = op['annual_average_revenue'].astype(float) * 1000
op['annual_average_gross_profit'] = op['annual_average_gross_profit'].astype(float) * 1000
colors = sns.color_palette('rocket')[0:5]
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plt1
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']

In [2]:
def plot_pie(plt10, plt11,  bi_title, **kwargs):
    """
    成功, 总和, 
    """
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(
        go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
        labels = ['succeed', 'fail']),
        1, 1)

    fig.add_trace(
        go.Pie(values = [plt10, plt11 - plt10], 
        labels = ['succeed', 'fail']),
        1, 2)
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_layout(title_text=bi_title,
    width=900,
    height=400,
    margin=dict(l=10, r=10, t=50, b=20),
    paper_bgcolor="LightSteelBlue",)

    return fig
 

In [17]:
plt10 = sum(df['contract_signed_opportunity_num'])  # win
plt11 = sum(df['total_opportunity_num'] - df['new_business_pipeline_unclosed_opp_num'])

# sum(df['total_opportunity_num'] - df['closed_canceled_opportunity_num'] - df['closed_lost_opportunity_num']) # 整体的商机成功率.a
# plot_pie(plt10, plt11,  "整体的商机本年成功率\t\n 成功数量")

In [ ]:
op['product1']

In [18]:
# ** 从年初到现在  总体 annual_average_revenue**
plt12 = df.query("total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum() # win
plt13 = df['annual_average_revenue'].sum() 
# plot_pie(plt12, plt13,  "整体的商机本年成功金额占比 \t\n 成功金额")

In [ ]:
df.head(3) # 今年所有客户数量 renew_opportunity_num

In [19]:
# plt20 = df.query("old_cust == 1 & if_cooped == 1 & total_opportunity_num > 0").shape[0] 
# plt21 = df.query("old_cust == 1 & total_opportunity_num > 0").shape[0] 
# # 0.592375366568915 旧客户留存率。
# plot_pie(plt20, plt21, ' \n\t 老客户新生意签约数量')
# # christine
# # 老客户新生意 *老客户老生意。*

In [28]:
# opportunity win & contract signed 区别.
plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['contract_signed_opportunity_num'].sum()
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")
plt31 = sum(plt31['total_opportunity_num'] - plt31['new_business_pipeline_unclosed_opp_num'])
# sum(plt31['total_opportunity_num'] - plt31['closed_canceled_opportunity_num'] - plt31['closed_lost_opportunity_num'])

# plot_pie(plt30, plt31,  "商机level: 已经合作过的客户本年商机成功率\t\n 成功数量")

In [27]:
# ** 从年初到现在老的客户为我们带来的新的商机的 annual_average_revenue 收益 **
plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum() 
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['annual_average_revenue'].sum()
# plot_pie(plt30, plt31,  "商机level: 已经合作过的客户本年成功金额占比 \t\n 成功金额")

In [ ]:
# df.query("old_cust == 0 & total_opportunity_num > 0 & ")['total_opportunity_num'].sum() 

In [25]:
# [df.query("old_cust == 1")['total_opportunity_num'].sum(), df.query("old_cust == 1")['total_opportunity_num'].sum() / sum(df['total_opportunity_num'])]
plt32 = df.query("old_cust == 0 & total_opportunity_num > 0")['contract_signed_opportunity_num'].sum() 
plt33 = df.query("old_cust == 0 & total_opportunity_num > 0") # xinke zong 
plt33 =   sum(plt33['total_opportunity_num'] - plt33['new_business_pipeline_unclosed_opp_num'])

# sum(plt33['total_opportunity_num'] - plt33['closed_canceled_opportunity_num'] - plt33['closed_lost_opportunity_num'])
  
# plot_pie(plt32, plt33, '商机level: 未合作过的客户本年商机成功率 \n\t 未合作过的本年商机成功数')
    # open 的要去掉

In [26]:
# ** 从年初到现在新增的客户为我们带来的 annual_average_revenue 收益 **
plt36 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum()
plt37 = df.query("old_cust == 0 & total_opportunity_num > 0")['annual_average_revenue'].sum() * .18# xinke zong 
# plot_pie(plt36, plt37, '商机level: 未合作过的本年成功金额占比 \t\n 成功金额')

In [ ]:
op['opportunity_record'].value_counts()
# 1089 na

In [ ]:
# # open 商机下, 前三十客户以及对应商机金额. 目前最应该重视的客户& 商机
# plt38 = op[op['milestone'].isin(mst_list) ]
# plt38['annual_average_gross_profit'] = plt38['annual_average_gross_profit'].astype(float)  
# plt38 = plt38.sort_values('annual_average_gross_profit', ascending=False).head(30)
# px.bar(plt38, x = 'account_name_en', y = 'annual_average_gross_profit')

In [ ]:
# op['opportunity_record']

In [30]:
# renew 型的商机; 到renew 期限的时候, 适当提醒.
plt39 = op.fillna(0).query("opportunity_record != 0 & opportunity_record != 'BCA Not Required'")
# plt39 = op[op['milestone'].isin(mst_list) ]
# plt39 = plt39[plt39['annual_average_gross_profit'].str.contains('[azAZ]').fillna(True)] 
plt39['annual_average_gross_profit'] = plt39['annual_average_gross_profit'].astype(float)
plt39 = plt39.sort_values('annual_average_gross_profit', ascending=False).head(30)
px.bar(plt39, x = 'account_name_en', y = 'annual_average_gross_profit', color = 'opportunity_record')


---
# 客户

In [ ]:
# 此前合作过的客户

plt42 = len(set(np.intersect1d(cus_bf_21, cus_af_21)))
plt43 = len(cus_af_21)

plot_pie(plt42, plt43, '客户level :  本年有商机的account占比 \n\t 老account数')
# """# [341, 717]  今年出现的过往的客户 vs 今年的所有"""

In [ ]:
# 此前未合作过的客户
plt40 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['customer_name'].nunique()
plt41 = df.query("old_cust == 0 & total_opportunity_num > 0")['customer_name'].nunique()
plot_pie(plt40, plt41, 'account level :  account转化率 \n\t account转化数')

# account

In [ ]:
# a0 = df['annual_average_revenue'].sum();a0 
# a1 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();a1
# b = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();b
# (a1 + b )/ a0 # 百分之四十的金额 被得到.

--- 
## ci milestone 和对应的contract value 的相关性

In [ ]:

op['prdct_list'] = op[['product1','product2', 'product3', 'product4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
op_new = op[op['createdon'] > '2020-11-01']
op_old = op[op['createdon'] <= '2020-11-01']

In [ ]:
op_new.shape

In [ ]:
newop = op_new[~op_new['milestone'].isin(mst_list)]

In [ ]:
newop[newop['prdct_list'].str.contains('(Value.added)') ]['annual_average_revenue'].sum()
# open vas ]

In [ ]:
op_new['prdct_list'].str.contains('(Value.added)').sum()

In [ ]:
sb_plot = pd.read_clipboard()

In [ ]:
px.sunburst()

In [ ]:
opp = op[['account_name_en','annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
opp = opp.fillna(0)
opp = opp[opp['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
opp = opp[opp['contract_term'] != 0] 
# opp = op
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']
sctr_list = ['Other', 0]
prodc_list = [0]

opp = opp[opp['milestone'].isin(mst_list)]
# opp = opp[~opp['sector'].isin(sctr_list)]
# opp = opp[~opp['product1'].isin(prodc_list)]

opp = opp[opp['annual_average_revenue'] != 0]
opp['annual_average_revenue_log'] = np.log(op['annual_average_revenue'])
px.scatter(opp, x = 'contract_term', y = 'annual_average_revenue_log',\
     color="milestone", trendline = 'ols')



` 1. milestone , 我们更容易签署周期更短的合同` <br>
` 2. 短期合同中, 我们赢得的合同比输掉的合同普遍价值更高. ` # ninahua

In [ ]:
# import statsmodels.formula.api as smf
# smf.ols('annual_average_revenue ~  milestone -1 ', data = op).fit().summary()


`商机金额大小与转化率的关系。 c 不显著#`

In [ ]:
# 商机金额大小与转化率的关系。 c
# df = df[(df['milestone'] != 'Early Lead') & (df['milestone'] != 'Potential Opportunity')]
def load_opp(df):
     df['win_or_loss'] = 0
     df['win_or_loss'] = df['win_or_loss'].where(df['milestone']  != 'Contract Signed', 1)
     df = df.fillna(0)
     # 删除有汉字的。
     df = df[~df['annual_average_gross_profit'].str.contains('[azAZ]').fillna(False)]
     df = df[~df['annual_average_revenue'].str.contains('[azAZ]').fillna(False)]
     return df
opp = load_opp(opp)
smf.logit(formula = "win_or_loss ~ contract_value ", data = opp).fit().summary()


In [ ]:
opp['milestone'].unique()

In [ ]:
# # violin
# px.violin(x = opp['win_or_loss'], y =  np.log(opp['contract_value']), \
#     points="all", color = opp['win_or_loss'], box = True )

### open shangji 
> open 商机可能带来 8743745010.0 的收入.

In [ ]:
# go.Figure(go.Sunburst(
#     labels=[ "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
#     parents=[  "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
#     values=[  14, 12, 10, 2, 6, 6, 4, 4],
# ))
opp = op[['account_name_en','annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
opp = opp.fillna(0)
opp = opp[opp['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
opp = opp[opp['contract_term'] != 0] 
opp[~opp['milestone'].isin(mst_list)]['annual_average_revenue'].astype(float).sum()
 

---
# sap

In [9]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
sap = pd.read_csv('./dws_dsc_customer_operation_monthly_kpi.csv', sep = '\001')
# op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
sap.columns = [re.sub('\w+\.', '', i) for i in list(sap.columns)] 
sap = sap.fillna(0)
sap['ytm_cogs'] = sap['ytm_income_amt'] - sap['ytm_gross_profit_amt']
sap['ytm_gp_margin'] = sap['ytm_gross_profit_amt'] / sap['ytm_income_amt'] 
sap['ytm_gp_margin'].replace([-np.inf, np.inf],  0, inplace = True)
sap['sap_lb_cost'] = sap[[
    'ytm_perm_hr_cost','ytm_service_outsourcing_amt','ytm_labor_service_outsourcing_amt',]].sum(axis = 1)
sap['customer_type'] = sap['customer_type'].replace('Q3', 'Q5').replace('Q4', 'Q3').replace('Q5', 'Q4')
sap= sap.query('ytm_gp_margin > -3 & ytm_gp_margin < 3 ') 
# df.columns 

# labor fee
# perm_hr_cost



In [ ]:
# # def plot_pie(plt10, plt11,  bi_title, **kwargs):
# #     """
# #     成功, 总和, 
# #     """
# #     colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
# #     fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
# #     fig.add_trace(
# #         go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
# #         labels = ['succeed', 'fail']),
# #         1, 1)

# #     fig.add_trace(
# #         go.Pie(values = [plt10, plt11 - plt10], 
# #         labels = ['succeed', 'fail']),
# #         1, 2)
# #     fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
# #                   marker=dict(colors=colors, line=dict(color='#000000', width=2)))
# #     fig.update_layout(title_text=bi_title,
# #     width=900,
# #     height=400,
# #     margin=dict(l=10, r=10, t=50, b=20),
# #     paper_bgcolor="LightSteelBlue",)

# #     return fig
# sap_ly = pd.concat([sap[['customer_name', 'bg_code', 'year', 'month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num']], 
#        sap[pd.Series(sap.columns)[pd.Series(sap.columns).str.contains('^ly')]]], axis = 1)
# sap_ly['year'] = 2020
# sap_ty = sap.iloc[:,0:28]
# sap_ly.columns = sap_ty.columns
# sap_long = pd.concat([sap_ty, sap_ly], axis = 0).drop(['month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num'], axis = 1)

In [10]:
sap_plt = sap[~sap['customer_name'].str.contains('ISC').fillna(False)]
sap_plt = sap_plt[~sap_plt['customer_name'].str.contains('Overhead').fillna(False)]
sap_plt = sap_plt[sap_plt['customer_name'] != 0]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
scaled = scaler.fit_transform(np.array(sap_plt['ytm_income_amt']).reshape(-1,1))
sap_plt['ytm_income_amt_scaled'] = np.log(scaled)
sap_plt['abs_ytm_income_amt'] = np.abs(sap_plt['ytm_income_amt'])
sap_plt= sap_plt.query('ytm_gp_margin > -.5 & ytm_gp_margin < .5 ') 
sap_plt['sap_lb_cost'] = np.abs(sap_plt['sap_lb_cost'])
# sap_plt = sap_plt.query("income_amt_scaled > -4")

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_39804/1364909686.py:7: RuntimeWarning:

divide by zero encountered in log



In [11]:
def bubble_plt(plot_data, x, color):
    if x == 'scaled':
        x = 'ytm_income_amt_scaled'
        vline = np.log(scaler.transform(np.array(10000000).reshape(-1,1)))[0][0]
    else:
        x = 'ytm_income_amt'
        vline = 10000000
    # y = y
    fig = px.scatter(
        data_frame= plot_data, x = x, y= 'ytm_gp_margin', hover_data=['customer_name'],
        size = 'sap_lb_cost', color = color, title='利润率y , 收入x, 人力成本 z, customer level')  
    
    fig.add_hline(y = .06, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
        annotation_text="2021 <br> GP_margin .06 <br><br />  " , 
        annotation_position="top right", annotation_opacity = .4, col = 1)
    fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
    return fig
# bubble_plt(sap_plt, 'scaled',  'customer_type')

`人力成本高中低，对应的客户，其利润率有无显著差异？说明什么?` <br>
`高人力成本 ~ 高利润 & 利润率.`

In [12]:
# bubble_plt(sap_plt, 'scaled', 'sap_lb_cost') 
sap_plt

,customer_name,bg_code,year,month,account_name_cn,geographical_location,industry,ou_num,inbound_line_count,inbound_receive_qty,outbound_line_count,outbound_shipped_qty,stock_turn_monthly,location_usage_rate,bms_bill_amount_exclude_tax,customs_agent_fee,operation_fee,warehouse_site_fee,short_barge_fee,labor_fee,equipment_usage_fee,value_added_fee,customs_agent_fee_rate,operation_fee_rate,warehouse_site_fee_rate,short_barge_fee_rate,labor_fee_rate,equipment_usage_fee_rate,value_added_fee_rate,fee_type,is_operation,has_vas,sap_start_month,income_amt,gross_profit_amt,customer_type,ytm_inbound_line_count,ytm_inbound_receive_qty,ytm_outbound_line_count,ytm_outbound_shipped_qty,ytm_stock_turn_monthly,ytm_location_usage_rate,ytm_bms_bill_amount_exclude_tax,ytm_customs_agent_fee,ytm_operation_fee,ytm_warehouse_site_fee,ytm_short_barge_fee,ytm_labor_fee,ytm_equipment_usage_fee,ytm_value_added_fee,ytm_income_amt,ytm_gross_profit_amt,ytm_gross_profit_rate,ytm_perm_hr_cost,ytm_service_outsourcing_amt,ytm_labor_service_outsourcing_amt,lm_inbound_line_count,lm_inbound_receive_qty,lm_outbound_line_count,lm_outbound_shipped_qty,ly_ytm_inbound_line_count,ly_ytm_inbound_receive_qty,ly_ytm_outbound_line_count,ly_ytm_outbound_shipped_qty,ly_ytm_stock_turn_monthly,ly_ytm_location_usage_rate,ly_ytm_bms_bill_amount_exclude_tax,ly_ytm_customs_agent_fee,ly_ytm_operation_fee,ly_ytm_warehouse_site_fee,ly_ytm_short_barge_fee,ly_ytm_labor_fee,ly_ytm_equipment_usage_fee,ly_ytm_value_added_fee,ly_ytm_income_amt,ly_ytm_gross_profit_amt,ly_ytm_gross_profit_rate,ly_ytm_perm_hr_cost,ly_ytm_service_outsourcing_amt,ly_ytm_labor_service_outsourcing_amt,ty_vs_ly_inbound_line_count,ty_vs_ly_inbound_receive_qty,ty_vs_ly_outbound_line_count,ty_vs_ly_outbound_shipped_qty,update_time,ytm_cogs,ytm_gp_margin,sap_lb_cost,ytm_income_amt_scaled,abs_ytm_income_amt
0,BURBERRY,CR,2021,202110,0.0,0.0,0.0,1,0.0,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,1.695769e+06,0.0,1.253202e+05,8.304081e+04,66358.760000,872397.669811,259697.358491,2.889543e+05,0.0,0.073902,0.048969,0.039132,0.514455,0.153144,0.170397,平衡型,非操作型,有VAS,201901,1675287.16,336320.86,Q1,0.0,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,1.301756e+07,0.0,9.693356e+05,1.021655e+06,6.635876e+04,6.951437e+06,1.504901e+06,2.503871e+06,1.653829e+07,3744002.53,0.226384,5426405.55,1935630.77,1107117.70,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,1.585851e+06,0.0,1.025896e+05,3.866184e+04,0.0,6.947404e+05,4.485142e+05,3.013448e+05,1.350771e+07,3.132437e+06,2.318999e-01,4234542.09,1234938.41,999854.64,0.000000,0.000000,0.000000,0.000000,2021-12-07 20:03:57.0,1.279429e+07,0.226384,8469154.02,-3.103435,1.653829e+07
1,DHL Supply Chain,"Other,TD",2021,202110,0.0,0.0,0.0,2,0.0,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,平衡型,非操作型,无VAS,201901,757348.50,-1146.01,Q4,0.0,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.260328e+06,-8175.02,-0.001306,350.02,6.14,264.09,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,3.436599e+06,-2.328306e-10,-6.775031e-17,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,2021-12-07 20:03:57.0,6.268503e+06,-0.001306,620.25,-4.041182,6.260328e+06
2,MARS,"EM,CR",2021,202110,0.0,0.0,0.0,12,37823.0,2.153919e+09,103582.0,1.688957e+09,3.483647,0.379304,1.454355e+07,0.0,2.491479e+06,4.127786e+06,0.000000,0.000000,172932.518868,7.751353e+06,0.0,0.171312,0.283822,0.000000,0.000000,0.011891,0.532975,VAS型,非操作型,有VAS,201901,15550779.44,3188642.65,Q1,186751.0,1.355535e+10,1089282.0,1.444477e+10,9.527309,0.224973,1.058739e+08,0.0,1.810302e+07,3.920842e+07,0.000000e+00,2.464151e+04,1.768931e+06,4.676892e+07,1.248438e+08,12252362.80,0.0

In [ ]:
# causal inference.
# `高人力成本 ~ 高利润率 .`?
smf.ols('sap_lb_cost ~ customer_type', data = sap).fit().summary()


In [ ]:
# smf.ols('ytm_gp_margin ~ sap_lb_cost-1', data = sap).fit().summary() #labor_fee

合并财务以及商务

In [13]:
df_plt = df.merge(sap[['sap_lb_cost','has_vas', 'customer_type', \
    'customer_name', 'ytm_gp_margin', 'ytm_gross_profit_amt']], on = 'customer_name')

In [14]:
df

,customer_name,bg_code,year,month,account_name_cn,geographical_location,industry,total_opportunity_num,annual_average_revenue,annual_average_gross_profit,new_opportunity_num,renew_opportunity_num,contract_signed_opportunity_num,potential_opportunity_num,closed_canceled_opportunity_num,closed_lost_opportunity_num,early_lead_opportunity_num,qualified_opportunity_num,shortlisted_opportunity_num,opportunity_gain_opportunity_num,peer_review_opportunity_num,dci_conducted_opportunity_num,closed_opportunity_num_in_current_month,opportunity_created_in_1_month,opportunity_created_between_1_3_month,opportunity_created_between_3_6_month,opportunity_created_over_6_month,win_opportunity_num,lost_opportunity_num,transportation_opportunity_num,warehouse_opportunity_num,customs_opportunity_num,vas_opportunity_num,other_service_opportunity_num,new_business_pipeline_unclosed_opp_num,total_contract_num,transportation_contract_num,warehouse_contract_num,customs_contract_num,contract_in_service,contract_end_min_date,contract_end_max_date,opportunity_win_rate,opportunity_lost_rate,update_time,if_cooped,old_cust
0,AllRightReserve,HK,2021,202110,AllRightReserve,0.0,Other,1,300000.0,15000.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
1,China Mobile,EI,2021,202110,中国移动,0.0,"Other,Technology",13,19910670.0,623330.0,3,10,5,3,0,4,0,0,0,1,0,0,2,1,3,0,0,5,4,10,2,2,0,0,4,5,4,1,0,4,2021-06-30T00:00:00Z,2022-12-31T00:00:00Z,0.384615,0.307692,2021-12-07 20:26:36.0,1,True
2,Jiangyin Yuxiang Automobile Accessory,TSC,2021,202110,江阴市羽项汽车饰件有限公司,0.0,Automotive,1,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
3,"Kunshan Geek Plus Supply Chain Management Co.,...",CR,2021,202110,昆山极智嘉供应链管理有限公司,0.0,Technology,1,1824410.0,9690.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
4,RITTER SPORT INTERNATIONAL HOLDING GmbH,CR,2021,202110,上海瑞特阿福品牌管理有限公司,0.0,Consumer,1,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,Zailab,HK,2021,202110,0,0.0,Life Sciences & Healthcare,1,2400000.0,480000.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
713,TD Tech,TSC,2021,202110,鼎桥,0.0,Technology,1,800000.0,120000.0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
714,Liebherr Group,TSC,2021,202110,利勃海尔集团,0.0,Consumer,1,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,False
715,OGAWA,CR,2021,202110,奥佳华按摩椅,0.0,Consumer,1,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000,0.000000,2021-12-07 20:26:36.0,0,True


In [ ]:
df_plt['abs_sap_lb_cost'] = np.abs(df_plt['sap_lb_cost'])

In [ ]:
# Q1到Q4不同类别的客户，商机成功率有无显著差异？
# 人力成本不同的客户，商机成功率有无显著差异？


# px.scatter(data_frame= df_plt, x = 'sap_lb_cost', y= 'opportunity_win_rate', hover_data=['customer_name'], 
#         size = 'abs_sap_lb_cost', color = 'customer_type', title='人力费用, 商机胜率, 客户类型')  



# px.violin(x = df_plt['customer_type'], y =  df_plt['sap_lb_cost'], \
#     points="all", color = df_plt['customer_type'], box = True )


    
# px.violin(x = df_plt['customer_type'], y =  df_plt['opportunity_win_rate'], \
#     points="all", color = df_plt['customer_type'], box = True )

In [ ]:
# 客户类型, / 人力成本 商机胜率

smf.ols('opportunity_win_rate ~   customer_type - 1 ', data = df_plt).fit().summary()
# smf.ols('opportunity_win_rate ~  sap_lb_cost  - 1 ', data = df_plt).fit().summary()

In [ ]:
# df_plt['sap_lb_cost'].describe()


In [ ]:
# 人力成本占比越高的业务，越可能产生高的毛利? yes
smf.ols('ytm_gross_profit_amt ~ sap_lb_cost-1 ', data = df_plt).fit().summary() #sap_lb_cost

In [ ]:
# p7

In [ ]:
# sap_plt['customer_type'].value_counts()

In [ ]:
sap['customer_name'].nunique()

In [ ]:
sap['customer_type'].value_counts()

In [ ]:
px.bar(data_frame= sap, x = 'customer_type',  hover_data=['customer_name'], 
    color = 'customer_type', title='customer type count')  

# Q4    133
# Q3     80
# 0      47
# Q1     46
# Q2     44

In [ ]:
# sap_plt.columns

In [ ]:
def bubble_plt(plot_data, x, color):
    if x == 'scaled':
        x = 'ytm_income_amt_scaled'
        vline = np.log(scaler.transform(np.array(10000000).reshape(-1,1)))[0][0]
    else:
        x = 'ytm_income_amt'
        vline = 10000000
    # y = y
    fig = px.scatter(
        data_frame= plot_data, x = x, y= 'ytm_gp_margin', hover_data=['customer_name'],
        size = 'abs_ytm_income_amt', color = color, title='利润率y , 收入x, 收入绝对值z customer level')  
    
    fig.add_hline(y = .06, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
        annotation_text="2021 <br> GP_margin .06 <br><br />  " , 
        annotation_position="top right", annotation_opacity = .4, col = 1)
    fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
    return fig
# bubble_plt(sap_plt, 'scaled',  'customer_type')

In [ ]:
# p9
bubble_plt(sap_plt, 'scaled', 'is_operation') # has_vas 
# bubble_plt(sap_plt, 'scaled', 'has_vas') #  
# df_plt['has_vas']

In [ ]:
# 不同仓储客户的分类
# 有无仓储业务，其利润率高低，有无显著差异？
# 不同仓储业务类型的客户，利润率有无显著差异？
# fee_type

smf.ols('ytm_gross_profit_amt ~ is_operation ', data = sap).fit().summary() #ytm_gross_profit_amt

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary() 

In [ ]:
corr_vas = sap['has_vas'].replace('有VAS', 1).replace('无VAS', 0)
corr_vas = pd.concat([corr_vas, sap['ytm_gross_profit_amt']], axis = 1)
coach_corr = corr_vas.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
corr_lb = sap[['sap_lb_cost', 'ytm_gross_profit_amt']]

In [ ]:
coach_corr = corr_lb.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
sap['is_operation'].unique()

In [ ]:
corr_opera = sap['is_operation'].replace('操作型', 1).replace('非操作型', 0)
corr_opera = pd.concat([corr_opera, sap['ytm_gross_profit_amt']], axis = 1)
coach_corr = corr_opera.corr()
coach_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:

# 不同象限WMS客户，其inbound/outbound量是否有显著区别？
smf.ols(
    'ytm_gross_profit_amt ~ inbound_line_count +\
         inbound_receive_qty + outbound_line_count + outbound_shipped_qty ' # + stock_turn_monthly + location_usage_rate
         , data = sap).fit().summary()

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary()